# Installing Dependencies

In [73]:
import sys

# Installing numpy
!{sys.executable} -m pip install numpy

# Installing pandas
!{sys.executable} -m pip install pandas

# Installing matplotlib
!{sys.executable} -m pip install matplotlib

# Installing statsmodels
!{sys.executable} -m pip install statsmodels

#Installing tqdm
!{sys.executable} -m pip install tqdm

#Installing Geopy
!{sys.executable} -m pip install geopy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports

In [74]:
# Import Statistic Libraries
import numpy                    as np
import pandas                   as pd
import matplotlib.pyplot        as plt

# Import Utility Libraries
from typing                     import List, Dict, TypedDict
from tqdm                       import tqdm
from matplotlib.lines           import Line2D
from geopy.geocoders            import Nominatim

# Algorythm

In [75]:
class UserRequest(TypedDict):
    City: str
    MinPrice: float
    MaxPrice: float
    MinSpace: float
    MaxSpace: float
    range: int

In [76]:
users_df: pd.DataFrame = pd.read_csv('../data/users.csv')
requests_df: pd.DataFrame  = pd.read_csv('../data/requests.csv')
listings_df: pd.DataFrame  = pd.read_csv('../data/combined_data.csv',delimiter=";")

listings_df['size'] = listings_df['size'].str.extract('(\d+)').astype(int)
listings_df['price'] = listings_df['price'].str.replace(',', '').astype(int)
listings_df['street'] = listings_df['street'].str.rstrip(',')

listings_df['start_date'] = listings_df['date'].str.extract(r'(?:From (\d+ [a-zA-Z]+ \d+))|(\d+ [a-zA-Z]+ \d+) -').bfill(axis=1).iloc[:, 0]
listings_df['end_date'] = listings_df['date'].str.extract(r' - (\d+ [a-zA-Z]+ \d+)').bfill(axis=1).iloc[:, 0]

listings_df['furnishing'] = listings_df['furnishing'].map({"unfurnished": False, "furnished": True})

listings_df['start_date'] = pd.to_datetime(listings_df['start_date'], errors='coerce', format='%d %b %Y')
listings_df['end_date'] = pd.to_datetime(listings_df['end_date'], errors='coerce', format='%d %b %Y')
listings_df = listings_df.drop('date', axis=1)

geolocator = Nominatim(user_agent="geo_locator")

def get_coordinates(row):
    location = geolocator.geocode(f"{row['street']}, {row['city']}")
    if location:
        return location.latitude, location.longitude
    else:
        return None, None
    
listings_df[['latitude', 'longitude']] = listings_df.apply(get_coordinates, axis=1, result_type='expand')

In [81]:
listings_df.head()

,street,city,size,furnishing,type,utilities,price,url,start_date,end_date,latitude,longitude
0,Galjoenstraat,Zaandam,16,True,Roomin a studenthouse,month,800,https://kamernet.nl/en/for-rent/room-zaandam/g...,2024-03-03,NaT,52.442913,4.845643
1,Stoofsteeg,Gouda,58,False,Apartment,month,1325,https://kamernet.nl/en/for-rent/apartment-goud...,2024-04-01,NaT,52.011768,4.711833
2,Beeklaan,Den Haag,17,True,Studio,month,725,https://kamernet.nl/en/for-rent/studio-den-haa...,2024-03-03,2026-03-01,52.074345,4.281000
3,Fuutlaan,Eindhoven,16,False,Room,month incl. utilities,295,https://kamernet.nl/en/for-rent/room-eindhoven...,2024-02-16,NaT,51.444204,5.501434
4,Meidoornsingel,Rotterdam,20,True,Room,month incl. utilities,925,https://kamernet.nl/en/for-rent/room-rotterdam...,2024-09-01,2025-03-01,51.953156,4.464757


In [103]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  

    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)

    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

def match_listings(user_request: pd.DataFrame, listings_df: pd.DataFrame):
    geolocator = Nominatim(user_agent="geo_locator")

    location = geolocator.geocode(user_request['city'])
    user_latitude, user_longitude = location.latitude, location.longitude

    matching_df = listings_df[
        (listings_df['furnishing']==user_request["furnishing"] if not pd.isna(user_request['furnishing']) else True) &
        ((listings_df["city"] == user_request["city"])
            | (haversine_distance(user_latitude, user_longitude, listings_df["latitude"], listings_df["longitude"]) < user_request["range"])) &
        (listings_df['price'] >= user_request['MinPrice'] if not pd.isna(user_request['MinPrice']) else True) &
        (listings_df['price'] <= user_request['MaxPrice'] if not pd.isna(user_request['MaxPrice']) else True) &
        (listings_df['size'] >= user_request['MinSpace'] if not pd.isna(user_request['MinSpace']) else True) &
        (listings_df['size'] <= user_request['MaxSpace']if not pd.isna(user_request['MaxSpace']) else True)
    ].copy()  

    matching_df['distance'] = matching_df.apply(lambda row: haversine_distance(user_latitude, user_longitude, row['latitude'], row['longitude']), axis=1)

    return matching_df

def rank_listings(matching_results):
    max_normalized_price = matching_results['price'].max() / matching_results['size'].max()
    max_distance = matching_results['distance'].max()
    max_regular_price = matching_results['price'].max()

    matching_results['normalized_price'] = (matching_results['price'] / matching_results['size']) / max_normalized_price
    matching_results['normalized_distance'] = matching_results['distance'] / max_distance
    matching_results['normalized_regular_price'] = matching_results['price'] / max_regular_price

    weight_price_per_m2 = 0.1
    weight_distance = 0.7
    weight_regular_price = 0.2
    
    matching_results['ranking_score'] = (
        (1 / matching_results['normalized_price']) * weight_price_per_m2 +
        (1 / (1 + matching_results['normalized_distance'])) * weight_distance +
        (1 / matching_results['normalized_regular_price']) * weight_regular_price
    )

    ranked_results = matching_results.sort_values(by='ranking_score', ascending=False)
    ranked_results = ranked_results.reset_index(drop=True)
    
    return ranked_results

In [107]:
user_request: UserRequest = requests_df.iloc[3].to_dict()
print(user_request)
matching_results = match_listings(user_request, listings_df)
ranked_results = rank_listings(matching_results)
print(len(ranked_results))

{'city': 'Rotterdam', 'MinPrice': 900.0, 'MaxPrice': nan, 'MinSpace': 9, 'MaxSpace': nan, 'range': 40, 'furnishing': nan, 'email': 'paarth@paarth.com'}
8


In [108]:
ranked_results[['street','city','size','furnishing','type','price','distance','url']]

,street,city,size,furnishing,type,price,distance,url
0,Graaf Balderikstraat,Rotterdam,28,False,Room,1000,0.398339,https://kamernet.nl/en/for-rent/room-rotterdam...
1,Spijkenissestraat,Rotterdam,65,True,Apartment,1100,3.430172,https://kamernet.nl/en/for-rent/apartment-rott...
2,Meidoornsingel,Rotterdam,20,True,Room,925,3.314682,https://kamernet.nl/en/for-rent/room-rotterdam...
3,Pleinweg,Rotterdam,85,True,Apartment,1300,3.638020,https://kamernet.nl/en/for-rent/apartment-rott...
4,Broersvest,Schiedam,40,True,Room,1000,5.263102,https://kamernet.nl/en/for-rent/room-schiedam/...
5,Admiraal de Ruyterweg,Rotterdam,61,True,Apartment,1800,0.662953,https://kamernet.nl/en/for-rent/apartment-rott...
6,Maaslandse Dam,Maasland,90,True,Apartment,1250,13.806962,https://kamernet.nl/en/for-rent/apartment-maas...
7,Stoofsteeg,Gouda,58,False,Apartment,1325,18.747017,https://kamernet.nl/en/for-rent/apartment-goud...
